In [ ]:
import os
import time

import torch

from apt.model import APTPredictor
from apt.utils import process_data

In [ ]:
device = "cuda:0"
checkpoint = "artifacts/saves/train-epoch-200-apt-regression/model_epoch=200.pt"

In [ ]:
state_dict, init_args = torch.load(checkpoint, map_location='cpu')
model = APTPredictor(**init_args)
model.load_state_dict(state_dict)
model.to(device)
model.eval()

APT(
  (_emb_x): FeedForward(
    (_emb): Linear(in_features=300, out_features=512, bias=True)
    (_net): Sequential(
      (0): GELU(approximate='none')
      (1): Linear(in_features=512, out_features=512, bias=True)
    )
  )
  (_emb_y): FeedForward(
    (_emb): Linear(in_features=1, out_features=512, bias=True)
    (_net): Sequential(
      (0): GELU(approximate='none')
      (1): Linear(in_features=512, out_features=512, bias=True)
    )
  )
  (_transformer): ModuleList(
    (0-11): 12 x TransformerBlock(
      (_attn): FullAttention(
        (to_qkv): Linear(in_features=512, out_features=1536, bias=False)
      )
      (_ln_attn): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (_ff): FeedForward(
        (_emb): Linear(in_features=512, out_features=1024, bias=True)
        (_net): Sequential(
          (0): GELU(approximate='none')
          (1): Linear(in_features=1024, out_features=512, bias=True)
        )
      )
      (_ln_ff): LayerNorm((512,), eps=1e-05, eleme

In [ ]:
save_dir = "results/"
for split in range(1, 6):
    data_dir = f"datasets/regression/OpenML-CTR23-{split}"
    datasets = os.listdir(data_dir)
    datasets = [dataset for dataset in datasets if ".pt" in dataset]

    with open(os.path.join(save_dir, f"apt-regression-{split}.csv"), "a") as f:
        f.write(','.join(["dataset", "mse", "mae", "r2", "time"]))
        f.write('\n')
        f.flush()
        for dataset in datasets:
            data = torch.load(os.path.join(data_dir, dataset), map_location='cpu')
            x_train, y_train, x_test, y_test = process_data(data["data"], classification=False)

            start_time = time.time()
            model.fit(x_train, y_train, process_data=False) # model.fit(x_train, y_train)
            metrics = model.evaluate(x_test, y_test)
            end_time = time.time()
            f.write(','.join([dataset] + [f'{val:5.4f}' for val in [metrics["Test MSE"], metrics["Test MAE"], metrics["Test R2"], end_time-start_time]]))
            f.write('\n')
            f.flush()